In [1]:
import numpy as np
import random
import networkx as nx
from matplotlib import pyplot as plt
import pandas as pd
import copy
import os
import sys
import networkx as nx
import seaborn as sns
sns.set(style='darkgrid')
sns.set_style(style='whitegrid')

In [2]:
# moduleー
current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append( str(current_dir) + '/../' )

In [3]:
from setting_param import MakeSample_prediction_num_of_edge_InputDir
from setting_param import L
InputDir = MakeSample_prediction_num_of_edge_InputDir

In [4]:
# READ EXIST_TABLE
EXIST_TABLE = np.load(InputDir + '/exist_table.npy')
EXIST_TABLE.shape

(3859, 68)

In [5]:
def ExistNodeList(ts):
    assert ts >= 0, "ts < 0 [referrence error]"
    return np.where(EXIST_TABLE[:, ts]==1)[0]

def GetAppearedNodes(ts):
    return set(ExistNodeList(ts)) - set(ExistNodeList(ts-1))

def GetObservedNodes(ts, L):
    U = set()
    for i in range(L):
        U |= set(ExistNodeList(ts-i))
    return U

def GetNodes(ts, L, node_type):
    if node_type=='all':
        node_set = set(ExistNodeList(ts))
    elif node_type=='stay':
        node_set = set(ExistNodeList(ts-1)) & set(ExistNodeList(ts))
    elif node_type=='lost':
        node_set = set(ExistNodeList(ts-1)) - set(ExistNodeList(ts))
    elif node_type=='return':
        node_set = GetAppearedNodes(ts) - (GetAppearedNodes(ts) - GetObservedNodes(ts-1, L))
    elif node_type=='new':
        node_set = GetAppearedNodes(ts) - GetObservedNodes(ts-1, L)
        node_set |= GetNodes(ts, L, 'return')
    return node_set

In [6]:
def Nx(ts):
    return  nx.from_numpy_matrix(np.load(InputDir + '/adjacency' + str(ts) + '.npy'))

def SubNxNew(ts, L):
    return nx.Graph(Nx(ts).edges(GetNodes(ts, L, 'new')))

def SubNxLost(ts, L):
    return nx.Graph(Nx(ts-1).edges(GetNodes(ts, L, 'lost')))

def GetEdges(ts, L, edge_type):
    G_1 = Nx(ts)
    if edge_type == "all":
        edge_set = G_1.edges
    elif edge_type == 'stay':
        G_0 = Nx(ts - 1)
        edge_set = G_0.edges & G_1.edges
    elif edge_type == "appeared":
        G_0 = Nx(ts - 1)
        edge_set = G_1.edges - G_0.edges - SubNxNew(ts, L).edges
    elif edge_type == "disappeared":
        G_0 = Nx(ts - 1)
        edge_set = G_0.edges - G_1.edges - SubNxLost(ts, L).edges
    return edge_set

In [7]:
def TsSplit(ts, L):
    ts_train = [(ts+l) for l in range(L)]
    ts_test = ts_train[-1]+1
    ts_all = ts_train.copy()
    ts_all.extend([ts_test])
    return ts_train, ts_test, ts_all

In [19]:
links = []
for ts in range(L, EXIST_TABLE.shape[1]):
    links.extend(list(GetEdges(ts, L, 'all')))
len(set(links))

94861

In [23]:
#  (0, 1return node)
n_links = []
for ts in range(L-1, EXIST_TABLE.shape[1]):
    n_links.extend(list(GetEdges(ts, L, 'all')))
len(set(n_links))

95203

In [24]:
#  (0, 1return node)
n_nodes = []
for ts in range(L-1, EXIST_TABLE.shape[1]):
    n_nodes.extend((GetNodes(ts, L, 'all')))
len(set(n_nodes))

3781

In [25]:
n_links = []
for ts in range(L-1, EXIST_TABLE.shape[1]):
    n_links.append(len(GetEdges(ts, L, 'all')))
n_links

[537,
 657,
 530,
 493,
 479,
 535,
 572,
 486,
 537,
 502,
 611,
 713,
 633,
 667,
 638,
 697,
 878,
 928,
 961,
 1281,
 1271,
 1260,
 1353,
 1429,
 1469,
 1842,
 1727,
 1663,
 1748,
 1884,
 2034,
 2038,
 1954,
 2091,
 2161,
 2307,
 2265,
 2350,
 2534,
 2607,
 2590,
 2664,
 2827,
 2883,
 2989,
 3176,
 3130,
 3138,
 3142,
 3162,
 3149,
 2979,
 3180,
 3392,
 3285,
 3284,
 3215,
 3107,
 3067,
 3200,
 3626,
 3471,
 3659,
 3823,
 3572,
 3719]

In [26]:
n_nodes = []
for ts in range(L-1, EXIST_TABLE.shape[1]):
    n_nodes.append(len(GetNodes(ts, L, 'all')))
n_nodes

[94,
 104,
 95,
 92,
 91,
 96,
 99,
 92,
 96,
 93,
 108,
 117,
 111,
 114,
 111,
 123,
 151,
 168,
 171,
 217,
 216,
 215,
 222,
 235,
 238,
 295,
 285,
 280,
 287,
 304,
 316,
 316,
 310,
 320,
 325,
 335,
 332,
 353,
 381,
 387,
 386,
 390,
 403,
 407,
 429,
 441,
 439,
 440,
 439,
 441,
 440,
 428,
 442,
 464,
 458,
 458,
 451,
 445,
 442,
 452,
 478,
 469,
 482,
 492,
 476,
 486]